In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from pydrake.math import sin, cos

%matplotlib inline

from pydrake.all import (SignalLogger, CompliantMaterial, ConstantVectorSource, DirectCollocation, DiagramBuilder, FloatingBaseType,
                         PiecewisePolynomial, RigidBodyTree, RigidBodyPlant,
                         SolutionResult, AddModelInstancesFromSdfString,
                         MathematicalProgram, Simulator, BasicVector, AddFlatTerrainToWorld)
from underactuated import (FindResource, PlanarRigidBodyVisualizer)

from dynamics import CubeVisualizer, cube_dynamics, ground_distances

import time
import random

In [2]:
np.linspace(0, -.5, 10)

array([ 0.        , -0.05555556, -0.11111111, -0.16666667, -0.22222222,
       -0.27777778, -0.33333333, -0.38888889, -0.44444444, -0.5       ])

In [2]:
vis = CubeVisualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7010/static/


In [41]:
def compute_optimal_control(initial_state,final_state, min_time, max_time, max_torque):

# Setup solver
    mp = MathematicalProgram()

# number knot points
    N = 50
    
# Total time used
    time_used = mp.NewContinuousVariables(1, "time_used") # add random variabe for time used
    dt = time_used/(N+1)
    
# Control decision variables
    u = mp.NewContinuousVariables(1, "u_%d" % 0) # only one input for the cube
    u_over_time = u
    for k in range(1,N):
        u = mp.NewContinuousVariables(1, "u_%d" % k)
        u_over_time = np.vstack((u_over_time, u))
    total_u = u_over_time
    
# Force decision variables
    f = mp.NewContinuousVariables(8, "f_%d" % 0) # only one input for the cube
    f_over_time = f
    for k in range(1,N):
        f = mp.NewContinuousVariables(8, "f_%d" % k)
        f_over_time = np.vstack((f_over_time, f))
    total_f = f_over_time

# State decision variables
    x = mp.NewContinuousVariables(14, "x_%d" % 0) # for both input thrusters
    x_over_time = x
    for k in range(1,N+1):
        x = mp.NewContinuousVariables(14, "x_%d" % k)
        x_over_time = np.vstack((x_over_time, x))
    total_x = x_over_time
    
# State Constraints
    error_thresh = 0.01
    for n in range(N):
        # Actual state vs. predicted (through dynamics) state
        state_next = total_x[n+1]
        dynamic_state_next = total_x[n,:] + cube_dynamics(total_x[n,:], total_u[n,:], total_f[n,:])*dt
        
        # Make sure the actual and predicted align to follow dynamics
        for j in range(14):
            state_error = state_next[j] - dynamic_state_next[j]
            mp.AddConstraint(state_error <= error_thresh)
            mp.AddConstraint(state_error >= -error_thresh)
            
# Can't penitrate the floor and can't leave the floor
    for n in range(N):
        distances = ground_distances(total_x[n,:])
        
        # the floor is at -.5
        floor_offset = -.01
        mp.AddConstraint(distances[0] >= floor_offset)
        mp.AddConstraint(distances[1] >= floor_offset)
        mp.AddConstraint(distances[2] >= floor_offset)
        mp.AddConstraint(distances[3] >= floor_offset)
        
        # don't leave the ground
        offset = 0.1
        mp.AddConstraint(distances[0] <= np.sqrt(2)+offset)
        mp.AddConstraint(distances[1] <= np.sqrt(2)+offset)
        mp.AddConstraint(distances[2] <= np.sqrt(2)+offset)
        mp.AddConstraint(distances[3] <= np.sqrt(2)+offset)
        
# Ground forces can't pull on the ground
    max_force = 100
    for n in range(N):
        force = total_f[n]
        for j in range(8):
            mp.AddConstraint(force[j] <= max_force)
            mp.AddConstraint(force[j] >= 0)
    
# add complimentary constraint
    for n in range(N):
        force = total_f[n]
        state = total_x[n]
        theta = state[4]
        
        distances = ground_distances(total_x[n+1,:])
        
        s = sin(theta)
        c = cos(theta)
        
        z_0 = force[0]*c + force[1]*s
        z_1 = - force[2]*s + force[3]*c
        z_2 = - force[4]*c - force[5]*s
        z_3 = force[6]*s - force[7]*c
        
        xy_0 = - force[0]*s + force[1]*c
        xy_1 = - force[2]*c - force[3]*s
        xy_2 = force[4]*s - force[5]*c
        xy_3 = force[6]*c + force[7]*s
        
        mu = 0.1
        
        mp.AddConstraint(xy_0 <= z_0*mu)
        mp.AddConstraint(xy_0 >= -z_0*mu)
        mp.AddConstraint(xy_1 <= z_1*mu)
        mp.AddConstraint(xy_1 >= -z_1*mu)
        mp.AddConstraint(xy_2 <= z_2*mu)
        mp.AddConstraint(xy_2 >= -z_2*mu)
        mp.AddConstraint(xy_3 <= z_3*mu)
        mp.AddConstraint(xy_3 >= -z_3*mu)
        
        
        vector_0 = force[0] * force[1]
        vector_1 = force[2] * force[3]
        vector_2 = force[4] * force[5]
        vector_3 = force[6] * force[7]
        
        val = np.asarray([vector_0, vector_1, vector_2, vector_3])

        thresh = 0.0001
        mp.AddConstraint(val.dot(distances) <= thresh)
        mp.AddConstraint(val.dot(distances) >= -thresh)
    
    
# cost
#     mp.AddQuadraticCost(1 * u_over_time[:,0].dot(u_over_time[:,0]))
#     for i in range(8):
#         mp.AddQuadraticCost(1 * f_over_time[:,i].dot(f_over_time[:,i]))


# initial state
    for i in range(14):
        # initial
        initial_state_error = x_over_time[0,i] - initial_state[i]
        mp.AddConstraint(initial_state_error == 0.0)

# don't care about final wheel angle
    error_thresh = 0.01
    for i in [0,1,2,3,4,5,7,8,9,10,11,12,13]:
        # final
        final_state_error = x_over_time[-1,i] - final_state[i]
        mp.AddConstraint(final_state_error <= error_thresh)
        mp.AddConstraint(final_state_error >= -error_thresh)
        
# direction of velocity constraint

#     for n in range(N):
#         state = total_x[n]
#         for i in range(14):
#             # vel of wheel only positive
#             mp.AddConstraint(state[-1] >= 0)
    
# add time constraint
    mp.AddConstraint(time_used[0] >= minimum_time)
    mp.AddConstraint(time_used[0] <= maximum_time)

# add torque constraints
    for n in range(N):
        mp.AddConstraint(u_over_time[n,0] <= max_torque)
        mp.AddConstraint(u_over_time[n,0] >= -max_torque)
        
        
# set the initial guess
    guess = np.linspace(initial_state[0], final_state[0], N)
    mp.SetInitialGuess(total_x[:][0], guess)
    

    print "Number of decision vars", mp.num_vars()
    print(mp.Solve())

    trajectory = mp.GetSolution(x_over_time)
    input_trajectory = mp.GetSolution(u_over_time)
    force_trajectory = mp.GetSolution(f_over_time)
    t = mp.GetSolution(time_used)
    time_array = np.arange(0.0, t, t/(N+1))
    
    return trajectory, input_trajectory, force_trajectory, time_array

In [42]:
m = MathematicalProgram()
m.SetInitialGuessForAllVariables

In [ ]:
np.linspace(0, -.5, 10)

In [43]:
# Set the inital condititions
x = 0.; z = 0.0; theta = 0.;
origin_1 = (x,0,z,0,theta,0,0,0,0,0,0,0,0,0.0)

x = -0.5; z = .5*(2**.5)-.5; theta = np.pi/4.0;
final_1 = (x,0,z,0,theta,0,0,0,0,0,0,0,0,0)

minimum_time = 0.5; maximum_time = 15.; max_torque = 1000.0


In [44]:
vis.draw_transformation(origin_1)

In [45]:
vis.draw_transformation(final_1)

In [ ]:
traj, input_traj, force_traj, time_array = \
compute_optimal_control(origin_1, final_1, minimum_time, maximum_time, max_torque)


In [10]:
# trajectory found
dt = time_array[1] - time_array[0]
for i in range(len(traj)):
    vis.draw_transformation(traj[i,:])
    time.sleep(dt)

In [ ]:
plt.plot(time_array, traj[:,-1])
plt.show()
plt.plot(time_array[:-1], input_traj[:])
plt.show()
plt.plot(time_array[:-1], force_traj[:])
plt.show()

In [24]:
angle = 0.0
angle_vel = 0.0 # 5.0
current_state = (0.,0.,0.0,0.0,angle,0.,0.,0.,0.,0.,0.,angle_vel,0.,0.)
vis.draw_transformation(current_state)

In [25]:
# code to simulate dynamics
sec = 0.5 # seconds
steps_per_sec = 1000.0 # discrete steps in a second
num_steps = sec*steps_per_sec # total number of steps
sec_per_step = sec/num_steps # number of seconds per step
for t in np.linspace(0,sec,num_steps):
    torque = [0.0]
    force = [0.,0.,0.,0.,0.,0.,0.,0.]
    current_state += cube_dynamics(current_state,torque,force)*sec_per_step
    vis.draw_transformation(current_state)
    time.sleep(sec_per_step)

/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  
